In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2
def load_and_preprocess_image(path):
    image = cv2.imread(path)
    image = cv2.resize(image, (224,224))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

In [2]:
# Useful functions for bilinear product
def outer_product(x):
    return K.batch_dot(x[0], x[1], axes = [1, 1]) / x[0].get_shape().as_list()[1]
def signed_sqrt(x):
    return K.sign(x)*K.sqrt(K.abs(x) + 1e-9)
def l2_normalize(x, axis = -1):
    return K.l2_normalize(x, axis=axis)

In [4]:
def insert_intermediate_layer_in_keras(model, layer_id, new_layer):
    from keras.models import Model

    layers = [l for l in model.layers]

    x = layers[0].output
    for i in range(1, len(layers)):
        if i == layer_id:
            x = new_layer(x)
        x = layers[i](x)

    model = Model(inputs=layers[0].input, outputs=x) ### inputs instead of input in this version
    return model

In [5]:
from tensorflow.keras.initializers import glorot_normal
from keras.layers import Lambda
import tensorflow

# Model Architecture
def VGG16_outerproduct(tensArry, classes, dropoutrate, model_name):
    K.clear_session()
    
    # VGG16 as base for extract features
#     base_model = VGG16(input_shape = (224, 224, 3), include_top = False, weights = "imagenet")
    
    tensor_input = keras.layers.Input(shape=[224,224,3])
    base_model = keras.applications.vgg16.VGG16(
                            input_tensor=tensor_input
                            , include_top=False
                            , weights='imagenet'
                        )
    
    
    def custom_layer_last(tensor):
        return tensor * tensArry
    lambda_layer = Lambda (custom_layer_last, name="lambda_New")

    base_model = insert_intermediate_layer_in_keras (base_model, 18, lambda_layer)
#     base_model.summary()
    
    # Get output from VGG16 end layer features
    feature_model_1 = base_model
    x1 = feature_model_1.layers[18].output
    x1_shape = feature_model_1.layers[18].output_shape
    
    feature_model_2 = base_model
    x2 = feature_model_2.layers[18].output
    x2_shape = feature_model_2.layers[18].output_shape
    
    # reshape to (batch_size, total_pixels, filter_size) for outer product
    x1 = Reshape(
        [x1_shape[1]*x1_shape[2], x1_shape[-1]])(x1)
    x2 = Reshape(
        [x2_shape[1]*x2_shape[2], x2_shape[-1]])(x2)
    
    # Outer product of VGG16 features
    x = Lambda(outer_product)([x1, x2])
    x = Reshape([x1_shape[-1]*x2_shape[-1]])(x)
    x = Lambda(signed_sqrt)(x)
    x = Lambda(l2_normalize)(x)
    
    # Final dense layer
    x = Dense(units=len(classes), kernel_initializer="glorot_uniform",
              kernel_regularizer=l2(1e-08),
             bias_initializer="glorot_uniform")(x)
    
    prediction = Activation(activation="softmax")(x)
    
    model = Model(inputs = base_model.input, outputs=prediction)
    
    ### added
    for layer in base_model.layers:
        layer.trainable = False
    ### added    
    return model

Using TensorFlow backend.


In [6]:
import scipy.io as sio

def load_data(path, data_type, data_class):
    raw = sio.loadmat(path)
    if data_class == "train":
        if data_type == "data":
            cars_data = [(row[5][0][:],row[4][0][0],row[0][0][0],row[1][0][0],row[2][0][0],row[3][0][0]) for row in raw['annotations'][0]]
            return cars_data

        if data_type == "class":
            cars_classes = [(row[0]) for row in raw['class_names'][0]]
            return cars_classes
        
    if data_class == "test":
        if data_type == "data":
            cars_data = [(row[4][0][:],"empty",row[0][0][0],row[1][0][0],row[2][0][0],row[3][0][0]) for row in raw['annotations'][0]]
            return cars_data


In [30]:
classes = load_data("devkit/cars_meta.mat", "class", "train")

In [32]:
from keras.layers import Lambda
from keras import backend as K
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, MaxPool2D, Reshape, Dense, Activation

import keras
from keras.regularizers import l2
from keras.models import Model, model_from_json, load_model, Sequential

# hyperparameters for initial training
lr = 0.1
epochs = 500
dropout_rate  = 0.5
batch_size = 64
model_name = "VGG19"
paitence = 10
optimizer_name = "sgd"
decay = 1e-8

# load model
tensArry=[]
for i in range(512):
    tensArry.append(1)
    
model = VGG16_outerproduct(tensArry, classes=classes,dropoutrate=dropout_rate,model_name=model_name)

In [33]:
# now all layers are trainable
for layer in model.layers:
    layer.trainable = True

# change LR
opt_sgd = keras.optimizers.SGD(
                lr=1e-3
                , decay=1e-9
                , momentum=0.9
                , nesterov=False
            )
model.compile(
    loss="categorical_crossentropy"
    , optimizer=opt_sgd
    , metrics=["categorical_accuracy"]
)

In [34]:
model.load_weights("final_all_transfer_learning_VGG16.h5")

In [35]:
layer = model.layers[18]
layer.name

'lambda_New'

In [36]:
classes = list(classes)
classes.sort()
classes

['AM General Hummer SUV 2000',
 'Acura Integra Type R 2001',
 'Acura RL Sedan 2012',
 'Acura TL Sedan 2012',
 'Acura TL Type-S 2008',
 'Acura TSX Sedan 2012',
 'Acura ZDX Hatchback 2012',
 'Aston Martin V8 Vantage Convertible 2012',
 'Aston Martin V8 Vantage Coupe 2012',
 'Aston Martin Virage Convertible 2012',
 'Aston Martin Virage Coupe 2012',
 'Audi 100 Sedan 1994',
 'Audi 100 Wagon 1994',
 'Audi A5 Coupe 2012',
 'Audi R8 Coupe 2012',
 'Audi RS 4 Convertible 2008',
 'Audi S4 Sedan 2007',
 'Audi S4 Sedan 2012',
 'Audi S5 Convertible 2012',
 'Audi S5 Coupe 2012',
 'Audi S6 Sedan 2011',
 'Audi TT Hatchback 2011',
 'Audi TT RS Coupe 2012',
 'Audi TTS Coupe 2012',
 'Audi V8 Sedan 1994',
 'BMW 1 Series Convertible 2012',
 'BMW 1 Series Coupe 2012',
 'BMW 3 Series Sedan 2012',
 'BMW 3 Series Wagon 2012',
 'BMW 6 Series Convertible 2007',
 'BMW ActiveHybrid 5 Sedan 2012',
 'BMW M3 Coupe 2012',
 'BMW M5 Sedan 2010',
 'BMW M6 Convertible 2010',
 'BMW X3 SUV 2012',
 'BMW X5 SUV 2007',
 'BMW X6

In [37]:
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
import numpy as np

def output(location, new_model):
    img = load_and_preprocess_image(location)
#     img = load_img(location, target_size = (224, 224, 3))
#     img = img_to_array(img)
#     img = img / 255
    img = np.expand_dims(img, axis=0)
    answer = new_model.predict(img)
    y_class = answer.argmax(axis = -1)
    
    top_3 = np.argsort(answer[0])[:-4:-1]
    # for i in range(3):
    #     print(" ({:.3})".format(answer[0][top_3[i]]))
    
    y = " ".join(str(x) for x in y_class)
    y = int(y)
    res = classes[y]
    indexClass = y
    # print(res)
#     print(" ({:.3})".format(answer[0][top_3[0]]))
    pred_prob = answer[0][top_3[0]]

    return float(pred_prob), res, indexClass

In [38]:
location = "data/testing/00001.jpg"
output(location, model)

(0.8746274709701538, 'Suzuki Aerio Sedan 2007', 180)

# Just as other architectures choose the top prototypes having highest confidence possible

In [51]:
# import cv2
# import os
# from shutil import copyfile
# from pathlib import Path

# #folder="/content/drive/My Drive/Colab/1KDataSet_Part1"
# folder = "data/training/"
# folder_Copy = "data/cars_classes_top/"

# last_fileName=""

# #images = []
# imagesFileNames = []
# for filename in os.listdir(folder):
#     # if filename not in os.listdir(folder_Copy):
#     for img in os.listdir(folder+filename):
#       if output(folder+filename+"/"+img, model)[0] >= 0.9 :
# #           print(folder+filename+"/"+img, model)
# #           print('path', folder+filename+"/"+img)
# #           print('prob=', output(folder+filename+"/"+img, model)[0])
#           src = folder+filename+"/"+img
#           dst = folder_Copy+output(folder+filename+"/"+img, model)[1]+"/"+img
#           if not os.path.isdir(folder_Copy+output(folder+filename+"/"+img, model)[1]):
#               print(folder+filename+"/"+img, model)
#               print('path', folder+filename+"/"+img)
#               print('prob=', output(folder+filename+"/"+img, model)[0])
#               Path(folder_Copy+output(folder+filename+"/"+img, model)[1]+"/").mkdir(parents=True, exist_ok=True)
#               copyfile(src, dst)
#               print(filename)
# #             print(img)
#           break;
#       #     img = cv2.imread(os.path.join(folder,filename))
#       #     if img is not None:
#       #         #images.append(img)
#           imagesFileNames.append(filename)
# imagesFileNames.sort()        
# # print(imagesFileNames[0])
# #images = load_images_from_folder(folder)


In [39]:
import ast
a_file = open("Filt_Resp_Cars_Crop.txt", "r")
# a_file = open("Gibbon_panda.txt", "r")
# a_file = open("Gimagenet_Test_2.txt", "r")


string_without_line_breaks = ""
for line in a_file:
    stripped_line = line.rstrip()
    string_without_line_breaks += stripped_line

import re
dict_Classes = ast.literal_eval(re.search('({.+})', string_without_line_breaks).group(0))

# for key, value in dict_Classes.items():
#     print ('classId', key)
#     print ('value', value)
a_file.close()
#print(dict_Classes)

In [40]:
import collections
od = collections.OrderedDict(sorted(dict_Classes.items()))
list_b=[]
for item in od:
    # print(item)
    list_b.append(item)

In [41]:
test_data = load_data("devkit/cars_test_annos_withlabels.mat", "data", "test")

In [42]:
test_data[2]

(array([145], dtype=uint8), 'empty', 51, 105, 968, 659)

In [43]:
st = str(test_data[3][0])
st

'[187]'

# ** Original Model**

In [18]:
import cv2
import os
from shutil import copyfile
from pathlib import Path
final_acc = 0
folder = "data/testing/"
imagesFileNames = []
i=0
correctClass = 0
#     print( len(os.listdir(folder+str(filename)) ))
for img in os.listdir(folder):
    if output(folder+"/"+img, model)[2] ==  test_data[i][0]-1:
        correctClass+=1
        print('image number: ' , i)
        print('correctly classified: ' , correctClass)
#         final_accu_by_class = correct_count_By_folder/len(os.listdir(folder))
#         print('correctly classified : ' ,filename, correct_count_By_folder)
#         print('accuracy for ' , filename , 'is ' , final_accu_by_class)
        print('final_acc : ', correctClass/8041) 
    i+=1
print('final_acc : ', correctClass/8041) 

#images = load_images_from_folder(folder)


image number:  0
correctly classified:  1
final_acc :  0.00012436264146250465
image number:  1
correctly classified:  2
final_acc :  0.0002487252829250093
image number:  2
correctly classified:  3
final_acc :  0.000373087924387514
image number:  3
correctly classified:  4
final_acc :  0.0004974505658500186
image number:  4
correctly classified:  5
final_acc :  0.0006218132073125233
image number:  5
correctly classified:  6
final_acc :  0.000746175848775028
image number:  6
correctly classified:  7
final_acc :  0.0008705384902375327
image number:  7
correctly classified:  8
final_acc :  0.0009949011317000372
image number:  8
correctly classified:  9
final_acc :  0.001119263773162542
image number:  9
correctly classified:  10
final_acc :  0.0012436264146250466
image number:  11
correctly classified:  11
final_acc :  0.0013679890560875513
image number:  12
correctly classified:  12
final_acc :  0.001492351697550056
image number:  13
correctly classified:  13
final_acc :  0.001616714339012

image number:  132
correctly classified:  106
final_acc :  0.013182439995025495
image number:  133
correctly classified:  107
final_acc :  0.013306802636487999
image number:  135
correctly classified:  108
final_acc :  0.013431165277950503
image number:  136
correctly classified:  109
final_acc :  0.013555527919413009
image number:  137
correctly classified:  110
final_acc :  0.013679890560875513
image number:  139
correctly classified:  111
final_acc :  0.013804253202338017
image number:  140
correctly classified:  112
final_acc :  0.013928615843800523
image number:  142
correctly classified:  113
final_acc :  0.014052978485263027
image number:  144
correctly classified:  114
final_acc :  0.014177341126725532
image number:  145
correctly classified:  115
final_acc :  0.014301703768188036
image number:  146
correctly classified:  116
final_acc :  0.01442606640965054
image number:  147
correctly classified:  117
final_acc :  0.014550429051113046
image number:  149
correctly classified: 

image number:  274
correctly classified:  209
final_acc :  0.025991792065663474
image number:  276
correctly classified:  210
final_acc :  0.02611615470712598
image number:  277
correctly classified:  211
final_acc :  0.026240517348588482
image number:  278
correctly classified:  212
final_acc :  0.02636487999005099
image number:  279
correctly classified:  213
final_acc :  0.026489242631513494
image number:  280
correctly classified:  214
final_acc :  0.026613605272975998
image number:  281
correctly classified:  215
final_acc :  0.026737967914438502
image number:  283
correctly classified:  216
final_acc :  0.026862330555901006
image number:  285
correctly classified:  217
final_acc :  0.026986693197363514
image number:  286
correctly classified:  218
final_acc :  0.027111055838826018
image number:  287
correctly classified:  219
final_acc :  0.02723541848028852
image number:  289
correctly classified:  220
final_acc :  0.027359781121751026
image number:  290
correctly classified:  2

image number:  416
correctly classified:  313
final_acc :  0.03892550677776396
image number:  417
correctly classified:  314
final_acc :  0.03904986941922647
image number:  418
correctly classified:  315
final_acc :  0.03917423206068897
image number:  419
correctly classified:  316
final_acc :  0.039298594702151475
image number:  420
correctly classified:  317
final_acc :  0.03942295734361398
image number:  421
correctly classified:  318
final_acc :  0.03954731998507648
image number:  422
correctly classified:  319
final_acc :  0.03967168262653899
image number:  423
correctly classified:  320
final_acc :  0.03979604526800149
image number:  424
correctly classified:  321
final_acc :  0.039920407909463995
image number:  425
correctly classified:  322
final_acc :  0.0400447705509265
image number:  426
correctly classified:  323
final_acc :  0.040169133192389
image number:  427
correctly classified:  324
final_acc :  0.040293495833851514
image number:  428
correctly classified:  325
final_

image number:  548
correctly classified:  417
final_acc :  0.051859221489864445
image number:  549
correctly classified:  418
final_acc :  0.05198358413132695
image number:  552
correctly classified:  419
final_acc :  0.05210794677278945
image number:  553
correctly classified:  420
final_acc :  0.05223230941425196
image number:  554
correctly classified:  421
final_acc :  0.05235667205571446
image number:  555
correctly classified:  422
final_acc :  0.052481034697176965
image number:  557
correctly classified:  423
final_acc :  0.052605397338639476
image number:  559
correctly classified:  424
final_acc :  0.05272975998010198
image number:  561
correctly classified:  425
final_acc :  0.052854122621564484
image number:  562
correctly classified:  426
final_acc :  0.05297848526302699
image number:  563
correctly classified:  427
final_acc :  0.05310284790448949
image number:  564
correctly classified:  428
final_acc :  0.053227210545951996
image number:  565
correctly classified:  429
f

...

image number:  7435
correctly classified:  5645
final_acc :  0.7020271110558388
image number:  7436
correctly classified:  5646
final_acc :  0.7021514736973014
image number:  7437
correctly classified:  5647
final_acc :  0.7022758363387638
image number:  7439
correctly classified:  5648
final_acc :  0.7024001989802263
image number:  7440
correctly classified:  5649
final_acc :  0.7025245616216889
image number:  7441
correctly classified:  5650
final_acc :  0.7026489242631514
image number:  7443
correctly classified:  5651
final_acc :  0.7027732869046138
image number:  7444
correctly classified:  5652
final_acc :  0.7028976495460764
image number:  7446
correctly classified:  5653
final_acc :  0.7030220121875389
image number:  7447
correctly classified:  5654
final_acc :  0.7031463748290013
image number:  7448
correctly classified:  5655
final_acc :  0.7032707374704639
image number:  7449
correctly classified:  5656
final_acc :  0.7033951001119264
image number:  7450
correctly classified

image number:  7558
correctly classified:  5748
final_acc :  0.7148364631264768
image number:  7559
correctly classified:  5749
final_acc :  0.7149608257679393
image number:  7562
correctly classified:  5750
final_acc :  0.7150851884094018
image number:  7563
correctly classified:  5751
final_acc :  0.7152095510508644
image number:  7564
correctly classified:  5752
final_acc :  0.7153339136923268
image number:  7565
correctly classified:  5753
final_acc :  0.7154582763337893
image number:  7567
correctly classified:  5754
final_acc :  0.7155826389752519
image number:  7568
correctly classified:  5755
final_acc :  0.7157070016167143
image number:  7571
correctly classified:  5756
final_acc :  0.7158313642581768
image number:  7573
correctly classified:  5757
final_acc :  0.7159557268996394
image number:  7574
correctly classified:  5758
final_acc :  0.7160800895411018
image number:  7576
correctly classified:  5759
final_acc :  0.7162044521825643
image number:  7577
correctly classified

image number:  7709
correctly classified:  5851
final_acc :  0.7276458151971148
image number:  7714
correctly classified:  5852
final_acc :  0.7277701778385773
image number:  7715
correctly classified:  5853
final_acc :  0.7278945404800398
image number:  7716
correctly classified:  5854
final_acc :  0.7280189031215023
image number:  7717
correctly classified:  5855
final_acc :  0.7281432657629648
image number:  7718
correctly classified:  5856
final_acc :  0.7282676284044273
image number:  7719
correctly classified:  5857
final_acc :  0.7283919910458898
image number:  7720
correctly classified:  5858
final_acc :  0.7285163536873523
image number:  7721
correctly classified:  5859
final_acc :  0.7286407163288148
image number:  7722
correctly classified:  5860
final_acc :  0.7287650789702773
image number:  7723
correctly classified:  5861
final_acc :  0.7288894416117399
image number:  7724
correctly classified:  5862
final_acc :  0.7290138042532024
image number:  7725
correctly classified

image number:  7857
correctly classified:  5954
final_acc :  0.7404551672677527
image number:  7858
correctly classified:  5955
final_acc :  0.7405795299092153
image number:  7859
correctly classified:  5956
final_acc :  0.7407038925506778
image number:  7860
correctly classified:  5957
final_acc :  0.7408282551921402
image number:  7861
correctly classified:  5958
final_acc :  0.7409526178336028
image number:  7862
correctly classified:  5959
final_acc :  0.7410769804750653
image number:  7863
correctly classified:  5960
final_acc :  0.7412013431165277
image number:  7864
correctly classified:  5961
final_acc :  0.7413257057579903
image number:  7865
correctly classified:  5962
final_acc :  0.7414500683994528
image number:  7866
correctly classified:  5963
final_acc :  0.7415744310409154
image number:  7867
correctly classified:  5964
final_acc :  0.7416987936823778
image number:  7869
correctly classified:  5965
final_acc :  0.7418231563238403
image number:  7870
correctly classified

image number:  7980
correctly classified:  6057
final_acc :  0.7532645193383908
image number:  7981
correctly classified:  6058
final_acc :  0.7533888819798532
image number:  7982
correctly classified:  6059
final_acc :  0.7535132446213157
image number:  7983
correctly classified:  6060
final_acc :  0.7536376072627783
image number:  7984
correctly classified:  6061
final_acc :  0.7537619699042407
image number:  7985
correctly classified:  6062
final_acc :  0.7538863325457033
image number:  7986
correctly classified:  6063
final_acc :  0.7540106951871658
image number:  7988
correctly classified:  6064
final_acc :  0.7541350578286283
image number:  7989
correctly classified:  6065
final_acc :  0.7542594204700908
image number:  7993
correctly classified:  6066
final_acc :  0.7543837831115533
image number:  7994
correctly classified:  6067
final_acc :  0.7545081457530158
image number:  7996
correctly classified:  6068
final_acc :  0.7546325083944783
image number:  7997
correctly classified

# > CauSim

In [28]:
values = dict_Classes.values()
values_list = list(values)
# print(values_list[int(test_data[0][0]-1)-1])

In [44]:
keysLst = list(dict_Classes.keys())

# for filename in os.listdir(folder):
for j in range(len(keysLst)):
    if classes[j] != keysLst[j]:
        print('index', j)
        print('found', classes[j], "different than",  keysLst[j])
        break;


In [45]:
import cv2
import os
from shutil import copyfile
from pathlib import Path
final_acc = 0
folder = "data/testing/"

k=0
correctClass = 0

for img in os.listdir(folder):
      print('image number: ' , k)
    #       print('class ', test_data[i][0]-1)
    #       print('key', )
      weights_dic_Conv2_13= {}
      value =   values_list[int(test_data[k][0]-1)]
#       print(value)
      weights_dic_Conv2_13.update(value)
      weights_dic_Conv2_13 = sorted (weights_dic_Conv2_13.items ())
      arryWeights_last_After = []
      od = collections.OrderedDict(weights_dic_Conv2_13)
      for i in range(len(od)):
          if weights_dic_Conv2_13[i][1][0] >0:
              arryWeights_last_After.append(1) 
          else:
              arryWeights_last_After.append(0)
      new_model = VGG16_outerproduct(arryWeights_last_After, classes=classes,dropoutrate=dropout_rate,model_name=model_name)

        # now all layers are trainable
      for layer in new_model.layers:
            layer.trainable = True

        # change LR
      opt_sgd = keras.optimizers.SGD(
                        lr=1e-3
                        , decay=1e-9
                        , momentum=0.9
                        , nesterov=False
                    )
      new_model.compile(
            loss="categorical_crossentropy"
            , optimizer=opt_sgd
            , metrics=["categorical_accuracy"]
        )
      new_model.load_weights('final_all_transfer_learning_VGG16.h5')
    #       break;
#       print(folder + img)
      print('*****************************')
      print(output(folder +img, new_model)[2])
      print(test_data[k][0]-1)
      print('*****************************')
      if output(folder +img, new_model)[2] ==  test_data[k][0]-1:
          correctClass+=1
          print('correctly classified: ' , correctClass)
      print('final_acc : ', correctClass/8041) 
      k+=1
      K.clear_session ()


image number:  0
*****************************
180
[180]
*****************************
correctly classified:  1
final_acc :  0.00012436264146250465
image number:  1
*****************************
102
[102]
*****************************
correctly classified:  2
final_acc :  0.0002487252829250093
image number:  2
*****************************
144
[144]
*****************************
correctly classified:  3
final_acc :  0.000373087924387514
image number:  3
*****************************
186
[186]
*****************************
correctly classified:  4
final_acc :  0.0004974505658500186
image number:  4
*****************************
184
[184]
*****************************
correctly classified:  5
final_acc :  0.0006218132073125233
image number:  5
*****************************
77
[77]
*****************************
correctly classified:  6
final_acc :  0.000746175848775028
image number:  6
*****************************
117
[117]
*****************************
correctly classified:  7
final_acc

*****************************
74
[74]
*****************************
correctly classified:  56
final_acc :  0.006964307921900261
image number:  57
*****************************
129
[129]
*****************************
correctly classified:  57
final_acc :  0.007088670563362766
image number:  58
*****************************
127
[127]
*****************************
correctly classified:  58
final_acc :  0.00721303320482527
image number:  59
*****************************
35
[35]
*****************************
correctly classified:  59
final_acc :  0.007337395846287775
image number:  60
*****************************
34
[34]
*****************************
correctly classified:  60
final_acc :  0.00746175848775028
image number:  61
*****************************
117
[117]
*****************************
correctly classified:  61
final_acc :  0.007586121129212784
image number:  62
*****************************
170
[170]
*****************************
correctly classified:  62
final_acc :  0.007710483

*****************************
169
[169]
*****************************
correctly classified:  112
final_acc :  0.013928615843800523
image number:  113
*****************************
32
[32]
*****************************
correctly classified:  113
final_acc :  0.014052978485263027
image number:  114
*****************************
163
[163]
*****************************
correctly classified:  114
final_acc :  0.014177341126725532
image number:  115
*****************************
153
[153]
*****************************
correctly classified:  115
final_acc :  0.014301703768188036
image number:  116
*****************************
45
[45]
*****************************
correctly classified:  116
final_acc :  0.01442606640965054
image number:  117
*****************************
60
[60]
*****************************
correctly classified:  117
final_acc :  0.014550429051113046
image number:  118
*****************************
8
[8]
*****************************
correctly classified:  118
final_acc :  0

106
[106]
*****************************
correctly classified:  163
final_acc :  0.02027111055838826
image number:  169
*****************************
87
[87]
*****************************
correctly classified:  164
final_acc :  0.020395473199850765
image number:  170
*****************************
42
[42]
*****************************
correctly classified:  165
final_acc :  0.02051983584131327
image number:  171
*****************************
159
[159]
*****************************
correctly classified:  166
final_acc :  0.020644198482775773
image number:  172
*****************************
146
[146]
*****************************
correctly classified:  167
final_acc :  0.020768561124238277
image number:  173
*****************************
24
[24]
*****************************
correctly classified:  168
final_acc :  0.020892923765700785
image number:  174
*****************************
104
[104]
*****************************
correctly classified:  169
final_acc :  0.02101728640716329
image nu

*****************************
148
[148]
*****************************
correctly classified:  216
final_acc :  0.026862330555901006
image number:  225
*****************************
78
[78]
*****************************
correctly classified:  217
final_acc :  0.026986693197363514
image number:  226
*****************************
91
[91]
*****************************
correctly classified:  218
final_acc :  0.027111055838826018
image number:  227
*****************************
87
[87]
*****************************
correctly classified:  219
final_acc :  0.02723541848028852
image number:  228
*****************************
32
[32]
*****************************
correctly classified:  220
final_acc :  0.027359781121751026
image number:  229
*****************************
111
[111]
*****************************
correctly classified:  221
final_acc :  0.02748414376321353
image number:  230
*****************************
72
[72]
*****************************
correctly classified:  222
final_acc :  0.

...

*****************************
132
[132]
*****************************
correctly classified:  7556
final_acc :  0.9396841188906853
image number:  7770
*****************************
39
[39]
*****************************
correctly classified:  7557
final_acc :  0.9398084815321477
image number:  7771
*****************************
183
[183]
*****************************
correctly classified:  7558
final_acc :  0.9399328441736102
image number:  7772
*****************************
172
[172]
*****************************
correctly classified:  7559
final_acc :  0.9400572068150728
image number:  7773
*****************************
86
[86]
*****************************
correctly classified:  7560
final_acc :  0.9401815694565353
image number:  7774
*****************************
68
[74]
*****************************
final_acc :  0.9401815694565353
image number:  7775
*****************************
183
[183]
*****************************
correctly classified:  7561
final_acc :  0.9403059320979977
imag

correctly classified:  7610
final_acc :  0.9463997015296605
image number:  7825
*****************************
18
[18]
*****************************
correctly classified:  7611
final_acc :  0.946524064171123
image number:  7826
*****************************
190
[190]
*****************************
correctly classified:  7612
final_acc :  0.9466484268125855
image number:  7827
*****************************
139
[139]
*****************************
correctly classified:  7613
final_acc :  0.946772789454048
image number:  7828
*****************************
46
[46]
*****************************
correctly classified:  7614
final_acc :  0.9468971520955105
image number:  7829
*****************************
162
[162]
*****************************
correctly classified:  7615
final_acc :  0.947021514736973
image number:  7830
*****************************
178
[179]
*****************************
final_acc :  0.947021514736973
image number:  7831
*****************************
158
[158]
****************

*****************************
39
[39]
*****************************
correctly classified:  7664
final_acc :  0.9531152841686358
image number:  7881
*****************************
110
[110]
*****************************
correctly classified:  7665
final_acc :  0.9532396468100982
image number:  7882
*****************************
169
[169]
*****************************
correctly classified:  7666
final_acc :  0.9533640094515607
image number:  7883
*****************************
170
[170]
*****************************
correctly classified:  7667
final_acc :  0.9534883720930233
image number:  7884
*****************************
144
[144]
*****************************
correctly classified:  7668
final_acc :  0.9536127347344857
image number:  7885
*****************************
45
[45]
*****************************
correctly classified:  7669
final_acc :  0.9537370973759483
image number:  7886
*****************************
131
[131]
*****************************
correctly classified:  7670
final_

correctly classified:  7718
final_acc :  0.959830866807611
image number:  7936
*****************************
19
[19]
*****************************
correctly classified:  7719
final_acc :  0.9599552294490735
image number:  7937
*****************************
31
[31]
*****************************
correctly classified:  7720
final_acc :  0.960079592090536
image number:  7938
*****************************
97
[97]
*****************************
correctly classified:  7721
final_acc :  0.9602039547319985
image number:  7939
*****************************
10
[8]
*****************************
final_acc :  0.9602039547319985
image number:  7940
*****************************
47
[47]
*****************************
correctly classified:  7722
final_acc :  0.960328317373461
image number:  7941
*****************************
167
[167]
*****************************
correctly classified:  7723
final_acc :  0.9604526800149236
image number:  7942
*****************************
16
[15]
************************

*****************************
68
[68]
*****************************
correctly classified:  7772
final_acc :  0.9665464494465863
image number:  7992
*****************************
60
[60]
*****************************
correctly classified:  7773
final_acc :  0.9666708120880487
image number:  7993
*****************************
165
[165]
*****************************
correctly classified:  7774
final_acc :  0.9667951747295113
image number:  7994
*****************************
118
[118]
*****************************
correctly classified:  7775
final_acc :  0.9669195373709738
image number:  7995
*****************************
118
[118]
*****************************
correctly classified:  7776
final_acc :  0.9670439000124362
image number:  7996
*****************************
160
[160]
*****************************
correctly classified:  7777
final_acc :  0.9671682626538988
image number:  7997
*****************************
185
[185]
*****************************
correctly classified:  7778
final_